In [1]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2023-10-19 13:48:32.469210: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 13:48:33.040816: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-19 13:48:33.622547: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-19 13:48:33.645969: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [3]:
tf.keras.backend.clear_session()

In [4]:
subject = 'São Paulo - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [5]:
data = pd.read_csv('2003_mo_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH,São Paulo - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,437.972720,0.314974,1.034903e+09,4.730940e+07,23.895559,9.819129e+08,8.310335,0.812723,782.111
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,434.363720,0.317328,1.036092e+09,4.732994e+07,23.901472,9.822666e+08,8.304194,0.812838,789.891
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,433.350868,0.319840,1.037280e+09,4.735047e+07,23.907385,9.826203e+08,8.298053,0.812953,774.309
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,431.825332,0.322104,1.038469e+09,4.737101e+07,23.913298,9.829741e+08,8.291912,0.813068,782.071
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,433.287298,0.324320,1.039658e+09,4.739155e+07,23.919210,9.833278e+08,8.285771,0.813183,841.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,515.306495,0.590649,NaN,NaN,NaN,NaN,NaN,NaN,1112.914
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514.812591,0.588569,NaN,NaN,NaN,NaN,NaN,NaN,1028.144
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514.856452,0.585196,NaN,NaN,NaN,NaN,NaN,NaN,996.449
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,515.215930,0.581094,NaN,NaN,NaN,NaN,NaN,NaN,1024.272


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
# scaler=MinMaxScaler()
# scaled_data = scaler.fit_transform(input_data)
# input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.410736,-1.192538,-1.718526,-0.426010,-1.485366,-2.737181,-0.804344,-2.274413
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.440931,-1.124056,-1.699080,-0.401643,-1.421647,-2.673768,-0.808200,-2.236038
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.449405,-1.050974,-1.679635,-0.377275,-1.357928,-2.610355,-0.812056,-2.197662
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.462169,-0.985091,-1.660189,-0.352908,-1.294210,-2.546942,-0.815911,-2.159286
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.449937,-0.920633,-1.640744,-0.328540,-1.230491,-2.483529,-0.819767,-2.120910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.209684,1.711139,1.107674,-1.587758,-1.453963,0.436685,1.040670,0.784878
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.192369,1.711746,1.098119,-1.580975,-1.459558,0.423023,1.034370,0.748669
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.178463,1.717654,1.088565,-1.574192,-1.465154,0.409361,1.028069,0.712461
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.166182,1.729082,1.079010,-1.567409,-1.470749,0.395699,1.021769,0.676253


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      706.731
1      697.361
2      859.304
3      762.222
4      763.920
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: São Paulo - Consumo de Cimento (t), Length: 240, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.410736,-1.192538,-1.718526,-0.426010,-1.485366,-2.737181,-0.804344,-2.274413
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.440931,-1.124056,-1.699080,-0.401643,-1.421647,-2.673768,-0.808200,-2.236038
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.449405,-1.050974,-1.679635,-0.377275,-1.357928,-2.610355,-0.812056,-2.197662
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.462169,-0.985091,-1.660189,-0.352908,-1.294210,-2.546942,-0.815911,-2.159286
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.449937,-0.920633,-1.640744,-0.328540,-1.230491,-2.483529,-0.819767,-2.120910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.608419,1.023937,1.193677,-1.393041,-1.206846,0.733610,1.413038,1.566441
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.653197,1.033712,1.195577,-1.411276,-1.217300,0.727991,1.392752,1.539489
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.699576,1.049101,1.197477,-1.429511,-1.227755,0.722372,1.372465,1.512536
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.757794,1.059564,1.199378,-1.447746,-1.238210,0.716753,1.352178,1.485583


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      706.731
1      697.361
2      859.304
3      762.222
4      763.920
        ...   
157    757.915
158    923.634
159    756.663
160    872.715
161    844.988
Name: São Paulo - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [11]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [12]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,1.357021,0.718372,0.835331,-0.070173,-0.126212,1.060884,0.965416,0.617332
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,1.320237,0.803494,0.853864,-0.120590,-0.162580,1.058281,0.995407,0.624883
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.259517,0.892135,0.872396,-0.171007,-0.198948,1.055678,1.025397,0.632435
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.196227,0.945408,0.890929,-0.221425,-0.235316,1.053074,1.055388,0.639987
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.135312,0.989786,0.909462,-0.271842,-0.271684,1.050471,1.085379,0.647538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.209684,1.711139,1.107674,-1.587758,-1.453963,0.436685,1.040670,0.784878
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.192369,1.711746,1.098119,-1.580975,-1.459558,0.423023,1.034370,0.748669
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.178463,1.717654,1.088565,-1.574192,-1.465154,0.409361,1.028069,0.712461
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.166182,1.729082,1.079010,-1.567409,-1.470749,0.395699,1.021769,0.676253


In [13]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [14]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [15]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[962742317, 1510830909, 551690126, 3640341245, 1473531527, 1124098331, 3820855234, 1671310039, 2708221203, 884791084, 3647402314, 4035436278, 317004277, 4029897697, 1186608720, 4116583551, 2259958206, 3465293327, 470846215, 4180386016, 1357016546, 3080750849, 38225298, 4244452675, 812965984, 899976137, 3772316090, 2449642851, 2971469748, 1680905249, 2809291457, 4292834578, 188995506, 1039506400, 1910751284, 2691522168, 4279819844, 3188541580, 3070962678, 1434863060, 106976327, 2724320014, 1583718857, 336270294, 4196153412, 540525676, 2389040015, 1213808413, 3967940630, 1839763408]


Step: 0 ___________________________________________


2023-10-19 13:48:33.979803: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-19 13:48:33.979998: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-19 13:48:33.980141: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

val_loss: 17093.697265625
winner_seed: 962742317


Step: 1 ___________________________________________
val_loss: 5822.8466796875
winner_seed: 1510830909


Step: 2 ___________________________________________
val_loss: 8411.966796875


Step: 3 ___________________________________________
val_loss: 8361.873046875


Step: 4 ___________________________________________
val_loss: 7055.61962890625


Step: 5 ___________________________________________
val_loss: 7575.072265625


Step: 6 ___________________________________________
val_loss: 10225.2841796875


Step: 7 ___________________________________________
val_loss: 18396.634765625


Step: 8 ___________________________________________
val_loss: 18599.34765625


Step: 9 ___________________________________________
val_loss: 7403.5302734375


Step: 10 ___________________________________________
val_loss: 16960.90234375


Step: 11 ___________________________________________
val_loss: 5462.6884765625
winner_seed: 4035436278


Step: 12 _____________

KeyboardInterrupt: 

In [18]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=4035436278)

Epoch 1/10000
4/4 [==============================] - 2s 139ms/step - loss: 916456.3750 - val_loss: 36806.8867
Epoch 2/10000
4/4 [==============================] - 0s 14ms/step - loss: 27037.6504 - val_loss: 18684.1289
Epoch 3/10000
4/4 [==============================] - 0s 13ms/step - loss: 27275.9141 - val_loss: 20223.0098
Epoch 4/10000
4/4 [==============================] - 0s 12ms/step - loss: 25557.2031 - val_loss: 35526.8555
Epoch 5/10000
4/4 [==============================] - 0s 13ms/step - loss: 26208.3848 - val_loss: 21128.0020
Epoch 6/10000
4/4 [==============================] - 0s 14ms/step - loss: 26267.1992 - val_loss: 19561.2598
Epoch 7/10000
4/4 [==============================] - 0s 13ms/step - loss: 24244.5938 - val_loss: 18117.3008
Epoch 8/10000
4/4 [==============================] - 0s 14ms/step - loss: 26356.2676 - val_loss: 18682.2148
Epoch 9/10000
4/4 [==============================] - 0s 13ms/step - loss: 26411.3516 - val_loss: 25259.8066
Epoch 10/10000
4/4 [======

4/4 [==============================] - 0s 11ms/step - loss: 42720.4102 - val_loss: 21696.8789
Epoch 77/10000
4/4 [==============================] - 0s 11ms/step - loss: 30174.4316 - val_loss: 26712.6582
Epoch 78/10000
4/4 [==============================] - 0s 11ms/step - loss: 31944.0586 - val_loss: 31603.4590
Epoch 79/10000
4/4 [==============================] - 0s 10ms/step - loss: 32656.2109 - val_loss: 19767.3809
Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 36667.8867 - val_loss: 23915.1484
Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 34232.0000 - val_loss: 26570.3984
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 60188.2461 - val_loss: 19546.4082
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 36022.4219 - val_loss: 26633.7559
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 42343.5430 - val_loss: 21073.5586
Epoch 85/10000
4/4 [==============

Epoch 151/10000
4/4 [==============================] - 0s 14ms/step - loss: 18637.1562 - val_loss: 13892.2021
Epoch 152/10000
4/4 [==============================] - 0s 13ms/step - loss: 12788.4189 - val_loss: 14907.9141
Epoch 153/10000
4/4 [==============================] - 0s 13ms/step - loss: 15627.3330 - val_loss: 14960.8096
Epoch 154/10000
4/4 [==============================] - 0s 13ms/step - loss: 17134.0352 - val_loss: 14719.9746
Epoch 155/10000
4/4 [==============================] - 0s 13ms/step - loss: 15150.9385 - val_loss: 13230.9688
Epoch 156/10000
4/4 [==============================] - 0s 14ms/step - loss: 16784.5840 - val_loss: 13916.6016
Epoch 157/10000
4/4 [==============================] - 0s 13ms/step - loss: 14958.0234 - val_loss: 13891.4639
Epoch 158/10000
4/4 [==============================] - 0s 13ms/step - loss: 13386.0049 - val_loss: 12396.7549
Epoch 159/10000
4/4 [==============================] - 0s 13ms/step - loss: 15732.9609 - val_loss: 12947.5420
Epoch 160/

4/4 [==============================] - 0s 14ms/step - loss: 14674.5674 - val_loss: 13831.6621
Epoch 226/10000
4/4 [==============================] - 0s 14ms/step - loss: 15748.8320 - val_loss: 13522.6924
Epoch 227/10000
4/4 [==============================] - 0s 14ms/step - loss: 14414.3730 - val_loss: 14166.9082
Epoch 228/10000
4/4 [==============================] - 0s 12ms/step - loss: 17115.4824 - val_loss: 14162.9229
Epoch 229/10000
4/4 [==============================] - 0s 13ms/step - loss: 12667.9453 - val_loss: 12658.0459
Epoch 230/10000
4/4 [==============================] - 0s 13ms/step - loss: 17372.4922 - val_loss: 12177.9795
Epoch 231/10000
4/4 [==============================] - 0s 13ms/step - loss: 13791.3477 - val_loss: 11950.0400
Epoch 232/10000
4/4 [==============================] - 0s 13ms/step - loss: 18231.6250 - val_loss: 12182.6230
Epoch 233/10000
4/4 [==============================] - 0s 13ms/step - loss: 16029.3770 - val_loss: 13422.6816
Epoch 234/10000
4/4 [=====

4/4 [==============================] - 0s 13ms/step - loss: 12429.2402 - val_loss: 12725.3037
Epoch 300/10000
4/4 [==============================] - 0s 12ms/step - loss: 10686.4287 - val_loss: 11228.3066
Epoch 301/10000
4/4 [==============================] - 0s 13ms/step - loss: 14287.1123 - val_loss: 13019.1074
Epoch 302/10000
4/4 [==============================] - 0s 13ms/step - loss: 12322.7412 - val_loss: 11769.2959
Epoch 303/10000
4/4 [==============================] - 0s 11ms/step - loss: 11894.0996 - val_loss: 11101.6045
Epoch 304/10000
4/4 [==============================] - 0s 12ms/step - loss: 12322.5342 - val_loss: 11033.7793
Epoch 305/10000
4/4 [==============================] - 0s 12ms/step - loss: 14462.9219 - val_loss: 10776.5264
Epoch 306/10000
4/4 [==============================] - 0s 13ms/step - loss: 11662.7305 - val_loss: 10109.9355
Epoch 307/10000
4/4 [==============================] - 0s 13ms/step - loss: 12045.1885 - val_loss: 9553.2422
Epoch 308/10000
4/4 [======

...

4/4 [==============================] - 0s 12ms/step - loss: 7691.6250 - val_loss: 9901.2031
Epoch 2082/10000
4/4 [==============================] - 0s 13ms/step - loss: 9634.1670 - val_loss: 10787.6592
Epoch 2083/10000
4/4 [==============================] - 0s 12ms/step - loss: 9655.4785 - val_loss: 10651.0830
Epoch 2084/10000
4/4 [==============================] - 0s 11ms/step - loss: 9074.4307 - val_loss: 10993.8662
Epoch 2085/10000
4/4 [==============================] - 0s 12ms/step - loss: 11058.0498 - val_loss: 11189.4580
Epoch 2086/10000
4/4 [==============================] - 0s 13ms/step - loss: 8028.6353 - val_loss: 10289.9746
Epoch 2087/10000
4/4 [==============================] - 0s 13ms/step - loss: 9354.3398 - val_loss: 10456.8486
Epoch 2088/10000
4/4 [==============================] - 0s 12ms/step - loss: 9451.0283 - val_loss: 9354.7568
Epoch 2089/10000
4/4 [==============================] - 0s 13ms/step - loss: 8896.7832 - val_loss: 10759.6250
Epoch 2090/10000
4/4 [======

4/4 [==============================] - 0s 10ms/step - loss: 9235.0391 - val_loss: 13411.8662
Epoch 2156/10000
4/4 [==============================] - 0s 11ms/step - loss: 9375.4385 - val_loss: 12497.9824
Epoch 2157/10000
4/4 [==============================] - 0s 11ms/step - loss: 8539.8740 - val_loss: 12078.8242
Epoch 2158/10000
4/4 [==============================] - 0s 10ms/step - loss: 10625.7695 - val_loss: 11368.0205
Epoch 2159/10000
4/4 [==============================] - 0s 10ms/step - loss: 9342.7549 - val_loss: 10729.1934
Epoch 2160/10000
4/4 [==============================] - 0s 10ms/step - loss: 8932.7480 - val_loss: 12054.7695
Epoch 2161/10000
4/4 [==============================] - 0s 10ms/step - loss: 8671.7783 - val_loss: 11165.3203
Epoch 2162/10000
4/4 [==============================] - 0s 11ms/step - loss: 8243.5049 - val_loss: 12194.0645
Epoch 2163/10000
4/4 [==============================] - 0s 11ms/step - loss: 8285.1982 - val_loss: 12231.9297
Epoch 2164/10000
4/4 [====

4/4 [==============================] - 0s 13ms/step - loss: 9005.7881 - val_loss: 10623.9482
Epoch 2230/10000
4/4 [==============================] - 0s 12ms/step - loss: 10150.8037 - val_loss: 10999.6621
Epoch 2231/10000
4/4 [==============================] - 0s 11ms/step - loss: 10975.3535 - val_loss: 10819.2002
Epoch 2232/10000
4/4 [==============================] - 0s 11ms/step - loss: 9791.8232 - val_loss: 11303.8037
Epoch 2233/10000
4/4 [==============================] - 0s 12ms/step - loss: 8794.5166 - val_loss: 9954.8896
Epoch 2234/10000
4/4 [==============================] - 0s 13ms/step - loss: 7472.5322 - val_loss: 11684.5488
Epoch 2235/10000
4/4 [==============================] - 0s 12ms/step - loss: 9429.1064 - val_loss: 9917.3965
Epoch 2236/10000
4/4 [==============================] - 0s 12ms/step - loss: 8169.9072 - val_loss: 10148.5176
Epoch 2237/10000
4/4 [==============================] - 0s 12ms/step - loss: 9433.9355 - val_loss: 10030.0137
Epoch 2238/10000
4/4 [=====

4/4 [==============================] - 0s 12ms/step - loss: 9276.1943 - val_loss: 11843.0742
Epoch 2304/10000
4/4 [==============================] - 0s 12ms/step - loss: 9339.4980 - val_loss: 12210.5850
Epoch 2305/10000
4/4 [==============================] - 0s 12ms/step - loss: 8292.8115 - val_loss: 12402.9619
Epoch 2306/10000
4/4 [==============================] - 0s 13ms/step - loss: 9718.1709 - val_loss: 13608.4062
Epoch 2307/10000
4/4 [==============================] - 0s 12ms/step - loss: 10651.9619 - val_loss: 11049.0264
Epoch 2308/10000
4/4 [==============================] - 0s 12ms/step - loss: 9618.8125 - val_loss: 11768.6094
Epoch 2309/10000
4/4 [==============================] - 0s 12ms/step - loss: 10166.4531 - val_loss: 12406.6797
Epoch 2310/10000
4/4 [==============================] - 0s 12ms/step - loss: 9473.1348 - val_loss: 13552.3838
Epoch 2311/10000
4/4 [==============================] - 0s 13ms/step - loss: 10331.9629 - val_loss: 12856.1328
Epoch 2312/10000
4/4 [==

4/4 [==============================] - 0s 13ms/step - loss: 9079.0049 - val_loss: 12719.2949
Epoch 2378/10000
4/4 [==============================] - 0s 13ms/step - loss: 9844.9863 - val_loss: 11822.9707
Epoch 2379/10000
4/4 [==============================] - 0s 13ms/step - loss: 8604.2803 - val_loss: 11534.0967
Epoch 2380/10000
4/4 [==============================] - 0s 11ms/step - loss: 10088.3516 - val_loss: 12046.5537
Epoch 2381/10000
4/4 [==============================] - 0s 13ms/step - loss: 9724.5381 - val_loss: 11406.8955
Epoch 2382/10000
4/4 [==============================] - 0s 13ms/step - loss: 9896.3262 - val_loss: 13105.3232
Epoch 2383/10000
4/4 [==============================] - 0s 13ms/step - loss: 9989.1396 - val_loss: 12932.4971
Epoch 2384/10000
4/4 [==============================] - 0s 13ms/step - loss: 9508.9688 - val_loss: 12881.1299
Epoch 2385/10000
4/4 [==============================] - 0s 13ms/step - loss: 8901.4590 - val_loss: 12838.6846
Epoch 2386/10000
4/4 [====

In [19]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [20]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,774.674622,866.304626,762.035889,755.123901,768.937622,821.974854,853.742493,846.420776,786.781982,860.49054,907.110718,816.958496,857.354736,836.677979,886.720276,857.656738,849.358154,835.065308,813.64447,800.848755,815.114014,809.130737,815.076782,831.78241,767.456848,865.358093,806.719727,885.945435,821.882996,778.507141
Target,910.653,937.696,871.611,795.853,804.767,712.733,763.665,727.397,855.245,855.042,648.867,1003.807,855.104,913.329,849.262,897.883,860.059,771.417,860.232,773.607,790.146,886.929,941.6,817.301,957.289,978.99,906.956,980.948,872.046,692.232
Error,135.978394,71.391357,109.575134,40.729126,35.829407,109.241882,90.077515,119.023804,68.463013,5.448547,258.243713,186.848511,2.250732,76.651001,37.458252,40.226257,10.700867,63.648315,46.587524,27.24176,24.968018,77.798279,126.523193,14.481384,189.832153,113.631897,100.236267,95.002563,50.163025,86.275146


In [21]:
display(mae)
display(mape)

80.48424

0.09624553

In [22]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [23]:
errors, mae, mape = year_mae_mape_calculator(trained_model,
                                             reshaped_test, 
                                             reshaped_test_target,
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 17ms/step
Ano-0: |Prediction[[9820.557]] - Target[9887.336000000001]| =  Error: [[66.7793]]; MAPE:[[0.00675402]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[10008.431]] - Target[10216.868999999999]| =  Error: [[208.43848]]; MAPE:[[0.0204014]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[4925.87]] - Target[5388.461]| =  Error: [[462.59082]]; MAPE:[[0.08584841]]


[array([[66.7793]], dtype=float32),
 array([[208.43848]], dtype=float32),
 array([[462.59082]], dtype=float32)]

245.9362

0.03766795